Inspired from this Kaggle notebook:
https://www.kaggle.com/code/kaerunantoka/h-m-eda-w-pyspark/notebook

In [1]:
## importing Python libraries

import numpy as np ##linear algebra
import pandas as pd ## data processing


In [2]:
## Starting spark session

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.memory", "15g").appName("EDA_spark").getOrCreate()

## Read Data

In [3]:
train = spark.read.option('header','true').csv("E:\\New journey\\Datasets\\HM\\transactions_train.csv")
article = spark.read.option('header','true').csv("E:\\New journey\\Datasets\\HM\\articles.csv")
customer = spark.read.option('header','true').csv("E:\\New journey\\Datasets\\HM\\customers.csv")



## Creating tables

In [4]:
train.createOrReplaceTempView("transactions_train")
article.createOrReplaceTempView("article")
customer.createOrReplaceTempView("customer")

## EDA with spark SQL

In [5]:
#taking a sample of 100
sample = spark.sql("select * from transactions_train limit 100")

In [6]:
train.columns

['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']

In [7]:
# Getting details on customer lists.
grouped= spark.sql('''          
                   select 
                       customer_id,
                       collect_list(article_id),
                       avg(price) as avg_price,
                       max(price) as max_price,
                       min(price) as min_price
                   
                   from transactions_train
                   group by customer_id
                   limit 100
                   
                   ''')


In [21]:
grouped.show()

+--------------------+------------------------+--------------------+--------------------+--------------------+
|         customer_id|collect_list(article_id)|           avg_price|           max_price|           min_price|
+--------------------+------------------------+--------------------+--------------------+--------------------+
|000346516dd355b40...|    [0534210011, 0666...| 0.02284322033898305|0.033881355932203386| 0.00676271186440678|
|0003e56a4332b2503...|            [0682520002]|0.024389830508474576|0.024389830508474576|0.024389830508474576|
|0011a72ff27917972...|    [0779059003, 0624...|0.037271186440677964| 0.08472881355932205|0.008457627118644067|
|0022058e10f379f15...|    [0846581002, 0846...|0.042355932203389825|0.042355932203389825|0.042355932203389825|
|0028449d82fdf6771...|    [0649437002, 0649...|0.033881355932203386|0.033881355932203386|0.033881355932203386|
|0037dbd391c8b1535...|    [0562480004, 0678...|0.035455205811138014|0.050830508474576264|0.025406779661016947|
|

In [19]:
# Changing to pandas dataframe.
df_grouped = grouped.toPandas()
df_grouped.shape

(100, 5)

In [22]:
## merging the dataframes

merged= spark.sql('''
    select * from transactions_train as tt
    left join customer as cs on cs.customer_id = tt.customer_id
    left join article as ar on ar.article_id = tt.article_id    
    limit 100

''')

In [24]:
print(merged.count())
print(merged.columns)

df_merged= merged.toPandas()
df_merged.head()

100
['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id', 'customer_id', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'article_id', 'product_code', 'prod_name', 'product_type_no', 'product_type_name', 'product_group_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code', 'colour_group_name', 'perceived_colour_value_id', 'perceived_colour_value_name', 'perceived_colour_master_id', 'perceived_colour_master_name', 'department_no', 'department_name', 'index_code', 'index_name', 'index_group_no', 'index_group_name', 'section_no', 'section_name', 'garment_group_no', 'garment_group_name', 'detail_desc']


,t_dat,customer_id,article_id,price,sales_channel_id,customer_id,FN,Active,club_member_status,fashion_news_frequency,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-11-14,1d355b0da25ff322f71d881e67f49e036c467a08914c66...,0511783004,0.02159322033898305,1,1d355b0da25ff322f71d881e67f49e036c467a08914c66...,1.0,1.0,ACTIVE,Regularly,...,None,None,None,None,None,None,None,None,None,None
1,2020-03-09,a0a542fc53c40b31ab6ca915a21f25c0388cb5b7b593fa...,0523490002,0.005067796610169492,1,a0a542fc53c40b31ab6ca915a21f25c0388cb5b7b593fa...,None,None,ACTIVE,NONE,...,None,None,None,None,None,None,None,None,None,None
2,2018-12-13,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,0565379022,0.008457627118644067,1,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,None,None,ACTIVE,NONE,...,None,None,None,None,None,None,None,None,None,None
3,2018-12-13,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,0565379022,0.008457627118644067,1,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,None,None,ACTIVE,NONE,...,None,None,None,None,None,None,None,None,None,None
4,2018-12-13,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,0565379022,0.008457627118644067,1,400aa157c34436031bad2cc4dbf7dc21f15e868f948f92...,None,None,ACTIVE,NONE,...,None,None,None,None,None,None,None,None,None,None
